This a notebook for COMP5349 Cloud Computing Assignment 2, implementing Spark Framework for data analyiss of online review data set released by Amazon on US sites. It provide various analytic phases to evaluate the data and calculate summary statistics, removing unwanted and meanning less data and finally performing some similarity analysis between review sentences.

The original data include product review information of Video DVD products from Amazon between 1995 and 2015. Each row reprenst one review given by one custoer on one prodcut and containing 15 columns of informaiton. This work load has only applied anaylisis based 5 columns of informaiton: customer_id, review_id, product_id, star_rating, review_body, while the additional information are striped out.

This notebook is designed to run on AWS EMR and obtain data from S3, after it's been downloaed into the bucket.

In [1]:
from pyspark.sql import SparkSession,dataframe
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType
from nltk.tokenize import sent_tokenize
import pyspark.sql.functions as fc
import numpy as np 
import tensorflow as tf
import tensorflow_hub as hub 
import scipy as sp 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1558506225954_0006,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
data = "s3://comp5349-ass2-wdai9162/amazon_reviews_us_Video_DVD_v1_00.tsv.gz"
schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("product_parent", IntegerType(), True),
    StructField("product_title",StringType(),True),
    StructField("product_category",StringType(),True),
    StructField("star_rating",IntegerType(),True),
    StructField("helpful_votes",IntegerType(),True),
    StructField("total_votes",IntegerType(),True),
    StructField("vine",StringType(),True),
    StructField("verified_purchase",StringType(),True),
    StructField("review_headline",StringType(),True),
    StructField("review_body",StringType(),False),
    StructField("review_date",StringType(),True)])


#"review_body" column has been set to not allowing for N/A type, due to possible error during tokenization 

VBox()

In [3]:
#read the data according to the defined schema
reviews = spark.read.csv(data,sep='\t',header=True, schema=schema)

#remove unrelated data column, keeping only customer_id, review_id, product_id, star_rating, review_body
#cache it to the default storage level 
clean_reviews = reviews.drop("marketplace") \
.drop("product_parent") \
.drop("product_title") \
.drop("product_category") \
.drop("helpful_votes") \
.drop("total_votes") \
.drop("vine") \
.drop("verified_purchase") \
.drop("review_headline") \
.drop("review_date") \
.repartition(3) \
.cache()

#.sample(True,0.5) \

#clean_reviews.count() #94s-1%data  

VBox()

In [4]:
#Stage 1.1 

#total number of reviews - approx 66.0s,73.0s -> 43s (1+3 m4.xlarge)
total = clean_reviews.rdd.count()
print ("the total number of reviews is %d" %(total))

spark.conf.set("spark.sql.shuffle.partitions", 20)

#the number of unique users - approx 84.0s , 76.0s -> 64s (1+3 m4.xlarge)
uniq_usr = clean_reviews.dropDuplicates(["customer_id"]).count()
print ("the number of unique users is %d" %(uniq_usr))

#the number of unique products - approx 72.1s -> 67s (1+3 m4.xlarge)
uniq_prdct = clean_reviews.dropDuplicates(["product_id"]).count()
print ("the number of unique products is %d" %(uniq_prdct))

VBox()

the total number of reviews is 5069140
the number of unique users is 2075970
the number of unique products is 297919
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52936)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 266, in handle
    poll(authenticate_and_accum_updates)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/l

In [5]:
#Stage 1.2 
spark.conf.set("spark.sql.shuffle.partitions", 20)

#the largest number of reviews published by a single user - approx 106.0s 
max_reviews_byUsr = clean_reviews.groupBy("customer_id").count().cache().agg({"count": "max"}).collect()
print ("the largest number of reviews published by a single user is %d" %(max_reviews_byUsr[0]))

#the top 10 users ranked by the number of reviews they publish - approx 83.0s 
sorted_list_user = clean_reviews.groupBy("customer_id").count().orderBy("count",ascending=[0]).show(10)
 
#the median number of reviews published by a user - approx 85.0s 
def calc_median(column):
    try:
        median = np.median(column) 
        return round(float(median),2)
    except Exception:
        return "input error" #return nothing if given value is wrong 

median_finder = fc.udf(calc_median,FloatType()) #define function for spark datafram 

review_list = clean_reviews.groupBy("customer_id").count().agg(fc.collect_list("count").alias("counts"))
review_list = review_list.withColumn("median",median_finder("counts")).cache()
review_list.show()

#store median for stage 2 use 
median_reviews_byUsr = int(review_list.select("median").collect()[0][0])

VBox()

the largest number of reviews published by a single user is 3582
+-----------+-----+
|customer_id|count|
+-----------+-----+
|   43430756| 3582|
|   18116317| 2987|
|   52287429| 2747|
|   52496677| 2650|
|   51110953| 2624|
|   19792742| 2495|
|   20018062| 2492|
|   50068216| 2379|
|   14539589| 2269|
|   50881246| 2104|
+-----------+-----+
only showing top 10 rows

+--------------------+------+
|              counts|median|
+--------------------+------+
|[24, 9, 107, 3, 1...|   1.0|
+--------------------+------+

In [6]:
#Stage 1.3 
spark.conf.set("spark.sql.shuffle.partitions", 20)

#the largest number of reviews written for a single product - approx 105.0s 
max_reviews_Product = clean_reviews.groupBy("product_id").count().agg({"count": "max"}).collect()
print ("the largest number of reviews written for a single product %d" %(max_reviews_Product[0]))
max_review = max_reviews_Product[0]

#the top 10 products ranked by the number of reviews they have - approx 84.0s 
sorted_products = clean_reviews.groupBy("product_id").count().orderBy("count",ascending=[0])
sorted_products.show(10)

#the median number of reviews a product has
product_list = clean_reviews.groupBy("product_id").count().agg(fc.collect_list("count").alias("counts"))
product_list = product_list.withColumn("median",median_finder("counts"))
product_list.show()
#store median for stage 2 use 
median_reviews_byPro = int(product_list.select("median").collect()[0][0])

VBox()

the largest number of reviews written for a single product 4969
+----------+-----+
|product_id|count|
+----------+-----+
|B00127RAJY| 4969|
|B0000AQS0F| 4967|
|B00G5G7K7O| 4439|
|B0002IQJ96| 4409|
|B00G5G7EXY| 4207|
|B003ZSJ212| 3641|
|B000X9FLKM| 3414|
|B000K8LV1O| 2989|
|B00N1JQ2UO| 2476|
|B00AMR5LZA| 2253|
+----------+-----+
only showing top 10 rows

+--------------------+------+
|              counts|median|
+--------------------+------+
|[162, 109, 57, 7,...|   3.0|
+--------------------+------+

In [7]:
#Stage 2.1 FILTER ONE - reviews with less than two sentences in the review body.
def custom_tokenize(text):
    if not text:
        text = ''
    return sent_tokenize(text)

#reviews with less than two sentences in the review_body - covert to RDD 
clean_reviews_filter_1 = clean_reviews.rdd.filter(lambda row:(len(custom_tokenize(row.review_body)))>=2).cache()
clean_reviews_filter_1.count()

VBox()

4016567

In [8]:
#Stage 2.2 FILTER TWO - reviews published by users with less than median number of reviews published
from pyspark.sql import Window
window = Window.partitionBy("customer_id")
clean_reviews_filter_2 = clean_reviews_filter_1.toDF().withColumn(colName = "review_count",col=fc.count("review_id").over(window)) \
.filter("review_count > %s" % median_reviews_byUsr).drop("review_count").cache()

clean_reviews_filter_2.count()

VBox()

2784806

In [9]:
#Stage 2.3 FILTER THREE - reviews from products with less than median number of reviews received
window = Window.partitionBy("product_id")
clean_reviews_filter_3 = clean_reviews_filter_2.withColumn(colName = "review_count_p",col=fc.count("review_id").over(window)) \
.filter("review_count_p > %s" % median_reviews_byPro).drop("review_count_p").cache()

clean_reviews_filter_3.count()

VBox()

2573240

In [10]:
#Stage 2.4 top 10 users ranked by median number of sentences in the reviews they provided  
clean_reviews_filter_3.rdd.map(lambda row:(row.customer_id,len(sent_tokenize(row.review_body)))). \
groupByKey().mapValues(sum).\
sortBy(lambda row:row[1],0).top(10,key=lambda row:row[1])


VBox()

[('12797924', 65317), ('19792742', 59591), ('51720265', 58834), ('49770892', 50601), ('50068216', 47156), ('38002140', 43163), ('50881246', 40991), ('50913245', 38952), ('52012289', 37770), ('52048190', 37042)]

In [25]:
#Stage 2.5 top 10 products ranked by median number of sentences in the reviews they have received 
clean_reviews_filter_3.rdd.map(lambda row:(row.product_id,len(sent_tokenize(row.review_body)))). \
groupByKey().mapValues(sum).\
sortBy(lambda row:row[1],0).top(10,key=lambda row:row[1])


VBox()

[('B0000AQS0F', 19756), ('B00127RAJY', 13981), ('B00003CXCT', 13846), ('B003ZSJ212', 13688), ('B00006HBUJ', 12621), ('B00003CX5P', 12509), ('B00005JMJ4', 11364), ('B00005JLXH', 9224), ('B00028HBKM', 8850), ('B008JFUPK8', 8772)]

In [12]:
#stage 3 

#pick the most popular prodcut by review number according to result calculated from stage 1.3 
pop_product_1st = sorted_products.select("product_id").collect()[0][0]
stage3_data = clean_reviews_filter_3.filter(clean_reviews_filter_3.product_id == pop_product_1st).cache()

positive_class = stage3_data.filter(stage3_data.star_rating >= 4)
positive_review = positive_class.select("review_body").rdd.flatMap(lambda x: x).collect()

negative_class = stage3_data.filter(stage3_data.star_rating <= 2)
negative_review = negative_class.select("review_body").rdd.flatMap(lambda x: x).collect()

print ("positive reviews of product %s = %d"%(pop_product_1st,len(positive_review)))
print ("negative reviews of product %s = %d"%(pop_product_1st,len(negative_review)))

VBox()

positive reviews of product B00127RAJY = 1521
negative reviews of product B00127RAJY = 73

In [13]:
#Utilzing Google pre-trained universal word embedding model to parse sentences to vectors 

module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
embed = hub.Module(module_url) 

#sentence segamentation for positive lists 
positive_review_sent = [] 

for i in range (len(positive_review)):
    positive_review_sent.append(sent_tokenize(positive_review[i]))
    
f_positive_sent = [y for x in positive_review_sent for y in x] #flat the list 

#sentence segamentation for negative lists 
negative_review_list = [] 
for i in range (len(negative_review)):
    negative_review_list.append(sent_tokenize(negative_review[i]))

f_negative_sent = [y for x in negative_review_list for y in x] #flat the list 

VBox()

In [14]:
#Use Google Pre-trained universal sentence encoder to encode each sentence to 512 dimension vector 
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    sentence_embeddings_pos = session.run(embed(f_positive_sent))
    sentence_embeddings_neg = session.run(embed(f_negative_sent))

VBox()

[None, None]

In [15]:
#5.2 Intra-Class Similarity, 0 = similar, 2 = different/opposite 
from scipy.spatial.distance import pdist,cdist

#calculate pairwise distances between observations in n-dimensional space.
dist_matrix_positive = pdist(sentence_embeddings_pos,"cosine")
dist_matrix_negative = pdist(sentence_embeddings_neg,"cosine")
dist_matrix_between = cdist(sentence_embeddings_pos,sentence_embeddings_neg,"cosine")


average_dist_positive = np.mean(dist_matrix_positive)
average_dist_negative = np.mean(dist_matrix_negative)

print (average_dist_positive)
print (average_dist_negative)

VBox()

0.6830594701765739
0.7267939128184078

In [16]:
np.mean(np.mean(dist_matrix_between,axis=1))

VBox()

0.7128082123923787

In [27]:
#Stage Four 6.1 
#prepare date by writing out to disk and read back in as Dataframe
from pyspark.mllib.feature import Word2Vec

import re 

#prepare data for model training and removing unnecessary regs 
positive_class_test = positive_class.select("review_body").rdd.map(lambda x: sent_tokenize(re.sub(r'&#[0-9]+;',' ',
                                                                          re.sub(r'<br />',' ', x[0]))))

                                                                          
negative_class_test = negative_class.select("review_body").rdd.map(lambda x: sent_tokenize(re.sub(r'&#[0-9]+;',' ',
                                                                          re.sub(r'<br />',' ', x[0]))))


model_Pos = Word2Vec().setVectorSize(100).setMinCount(1).setSeed(42).fit(positive_class_test)
model_Neg = Word2Vec().setVectorSize(100).setMinCount(1).setSeed(42).fit(negative_class_test)




VBox()

In [28]:
a = model_Pos.getVectors()

print (len(a))


VBox()

12688

In [29]:
b = model_Neg.getVectors()

print (len(b))

VBox()

646

In [30]:
pos_list=[]

for x in a:
    pos_list.append(a[x])
    

c=np.array(pos_list)
c.shape

VBox()

(12688, 100)

In [31]:
neg_list=[]

for x in b:
    neg_list.append(b[x])
    
d=np.array(neg_list)
d.shape

VBox()

(646, 100)

In [32]:
print (pdist(c),"cosine") 
print (np.mean(pdist(c)))

VBox()

[0.05080126 0.0471834  0.04869655 ... 0.04567042 0.03953254 0.04673391] cosine
0.044226288920317776

In [33]:
print (pdist(d),"cosine") 
print (np.mean(pdist(d)))

VBox()

[0.04298129 0.046965   0.03761173 ... 0.03929249 0.03910964 0.04232494] cosine
0.04080857673629429

In [34]:
print (cdist(c,d),"cosine")


VBox()

[[0.05180001 0.05176109 0.05240793 ... 0.05365713 0.05143519 0.04981803]
 [0.04169833 0.04355046 0.0426913  ... 0.04142708 0.04340602 0.04390789]
 [0.04208087 0.03955149 0.04095635 ... 0.03963504 0.04361656 0.04501934]
 ...
 [0.0410593  0.04164201 0.04258026 ... 0.04067823 0.04187385 0.0426368 ]
 [0.04423372 0.04225778 0.0415093  ... 0.03996423 0.04847304 0.04220427]
 [0.04311021 0.04463898 0.03813571 ... 0.03814442 0.04374516 0.0418509 ]] cosine